# 人机交互

人机交互（Human-in-the-loop, HITL）中间件用于在智能体执行过程中加入人工审批。

当触发人工审批时，HITL 中间件会[中断](https://reference.langchain.com/python/langgraph/types/#langgraph.types.interrupt)程序执行。此时，LangGraph 的状态保存在 [checkpointer](https://docs.langchain.com/oss/javascript/langgraph/persistence#checkpointer-libraries) 中，因此程序可以安全地暂停，并在人工审批之后恢复。

本文只是演示 checkpoint 在人机交互中的作用，无所谓线程结束后记忆是否保持，因此使用最方便的 `InMemorySaver`。在生产环节中，推荐使用由数据库支持的检查点，比如：

- `SqliteSaver`
- `PostgresSaver`
- `MongoDBSaver`
- `RedisSaver`

In [1]:
import os
import uuid
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

# 加载模型配置
_ = load_dotenv()

In [2]:
# 配置大模型服务
llm = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    model="qwen3-coder-plus",
)

# 工具函数
@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@tool
def add_numbers(a: float, b: float) -> float:
    """Add two numbers and return the sum."""
    return a + b

@tool
def calculate_bmi(weight_kg: float, height_m: float) -> float:
    """Calculate BMI given weight in kg and height in meters."""
    if height_m <= 0 or weight_kg <= 0:
        raise ValueError("height_m and weight_kg must be greater than 0.")
    return weight_kg / (height_m ** 2)

# 创建带工具调用的Agent
tool_agent = create_agent(
    model=llm,
    tools=[get_weather, add_numbers, calculate_bmi],
    middleware=[
        HumanInTheLoopMiddleware( 
            interrupt_on={
                # 无需触发人工审批
                "get_weather": False,
                # 需要审批，且允许approve,edit,reject三种审批类型
                "add_numbers": True,
                # 需要审批，允许approve,reject两种审批类型
                "calculate_bmi": {"allowed_decisions": ["approve", "reject"]},
            },
            description_prefix="Tool execution pending approval",
        ),
    ],
    checkpointer=InMemorySaver(),
    system_prompt="You are a helpful assistant",
)

In [3]:
# 运行Agent
config = {'configurable': {'thread_id': str(uuid.uuid4())}}
result = tool_agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "我身高180cm，体重180斤，我的BMI是多少"
        # "content": "what is the weather in sf"
    }]},
    config=config,
)

# result['messages'][-1].content
result.get('__interrupt__')

[Interrupt(value={'action_requests': [{'name': 'calculate_bmi', 'args': {'height_m': 1.8, 'weight_kg': 90}, 'description': "Tool execution pending approval\n\nTool: calculate_bmi\nArgs: {'height_m': 1.8, 'weight_kg': 90}"}], 'review_configs': [{'action_name': 'calculate_bmi', 'allowed_decisions': ['approve', 'reject']}]}, id='b58dbd2802c1ff4f89a7504b20d5d229')]

In [4]:
# Resume with approval decision
result = tool_agent.invoke(
    Command(
        resume={"decisions": [{"type": "approve"}]}  # or "edit", "reject"
    ), 
    config=config
)

result['messages'][-1].content

'您的BMI是27.8。根据中国成人BMI标准，这属于超重范围（BMI ≥ 24为超重，≥ 28为肥胖）。建议您关注饮食健康和适量运动，如有需要可咨询医生或营养师以获得更专业的建议。'

参考文档：

- [langchain/human-in-the-loop](https://docs.langchain.com/oss/python/langchain/human-in-the-loop)
- [langchain/short-term-memory](https://docs.langchain.com/oss/python/langchain/short-term-memory)
- [langchain/long-term-memory](https://docs.langchain.com/oss/python/langchain/long-term-memory)
- [langgraph/persistence](https://docs.langchain.com/oss/python/langgraph/persistence)
- [langgraph/use-time-travel](https://docs.langchain.com/oss/python/langgraph/use-time-travel)
- [langgraph/add-memory](https://docs.langchain.com/oss/python/langgraph/add-memory)